# 作成したモデルでの正誤を一覧にする

一旦モデル設計とのコードと分離したかったため、.h5ファイルを読み込んで使用する。

In [4]:
import os
# CPUの利用を強制する場合
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import shutil
import random
import tensorflow as tf
import keras
from keras import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.optimizers import SGD

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [5]:
# 環境変数とか

# 読み込むモデル
model_path = '.\\models\\vgg16-tuning-50epoch.h5'


# 元データ保存先
dataset_base_path = '.\\splat-scene-dataset'
dataset_split_base_path = '.\\dataset'
tensorboard_log_path = '.\\tflog'

# 画像設定
input_size = (640, 360)
input_shape = (640, 360, 3)

# データ関係
batch_size = 1 # 1回でいい
categories_n = 17

dataset_train_path = os.path.join(dataset_split_base_path, 'train')
dataset_val_path   = os.path.join(dataset_split_base_path, 'val')
dataset_test_path  = os.path.join(dataset_split_base_path, 'test')
pathes = [dataset_train_path, dataset_val_path, dataset_test_path]

In [6]:
def load_model(path):
    return keras.models.load_model(path)

model = load_model(model_path)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 640, 360, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 640, 360, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 640, 360, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 320, 180, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 320, 180, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 320, 180, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 160, 90, 128)      0         
__________

In [7]:
# ImageDataGeneratorを生成
def create_generator(path,
                     target_size = (640, 360),
                     batch_size = 1,
                     class_mode = 'categorical'):
    print(path)
    dg = ImageDataGenerator(rescale=1/255.0)
    gen = dg.flow_from_directory(path, 
                                 target_size=target_size,
                                 batch_size=batch_size,
                                 class_mode=class_mode,
                                 shuffle=False)
    return (dg, gen)

(train_dg, train_gen) = create_generator(dataset_train_path, target_size=input_size, batch_size=batch_size)
(val_dg,   val_gen)   = create_generator(dataset_val_path, target_size=input_size, batch_size=batch_size)
(test_dg,  test_gen)  = create_generator(dataset_test_path, target_size=input_size, batch_size=batch_size)

generators = [train_gen, val_gen, test_gen]

.\dataset\train
Found 10231 images belonging to 17 classes.
.\dataset\val
Found 3406 images belonging to 17 classes.
.\dataset\test
Found 3406 images belonging to 17 classes.


In [ ]:
predicts = [model.predict_generator(g, len(g.filenames), verbose=1) for g in generators]
predicts

 3862/10231 [==========>...................] - ETA: 2:15:04

In [ ]:
files = [g.filenames for g in generators]
results = [(f, p) for f,p in zip(files, predicts)]
results